In [ ]:
import pandas as pd
import numpy as np


import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [ ]:
import logging
logging.getLogger('tensorflow').disabled = True


In [ ]:
url = 'https://raw.githubusercontent.com/natashayulian/diamond_dataset/master/diamonds.csv'
df = pd.read_csv(url)

In [ ]:
df.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


### Split Dataset menjadi train, test, dan validation

In [ ]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

34521 train examples
8631 validation examples
10788 test examples


In [ ]:
#Membuat target dari prediksi model
# price 0 = low; 1 = high
df['target'] = np.where(df['price']==327, 0, 1)

# Drop un-used columns.
df = df.drop(columns=['price'])

### Create input pipeline using tf data

In [ ]:
# Cara untuk membuat dataset tf.data dari pandas dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = df.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [ ]:
batch_size = 10 #bath ukuran kecil untuk demonstrasi
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [ ]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of carat:', feature_batch['carat'])
  print('A batch of targets:', label_batch )

Every feature: ['Unnamed: 0', 'carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z']
A batch of carat: tf.Tensor([1.5  2.7  1.53 0.5  0.33 0.77 0.48 0.38 0.39 0.41], shape=(10,), dtype=float64)
A batch of targets: tf.Tensor([1 1 1 1 1 1 1 1 1 1], shape=(10,), dtype=int64)


In [ ]:
# We will use this batch to demonstrate several types of feature columns
example_batch = next(iter(train_ds))[0]

# A utility method to create a feature column
# and to transform a batch of data
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())


In [ ]:
#numeric column
carat = feature_column.numeric_column('carat')
demo(carat)

[[0.9 ]
 [0.42]
 [2.  ]
 [1.25]
 [0.32]
 [0.3 ]
 [0.3 ]
 [1.25]
 [1.08]
 [1.91]]


In [ ]:
#bucketized column 
carat = feature_column.numeric_column('carat')
carat_buckets = feature_column.bucketized_column(carat, boundaries=[1, 2])
demo(carat_buckets)

[[1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [ ]:
#categorical
color_type = feature_column.categorical_column_with_vocabulary_list(
      'color', ['E', 'I','J','D','H', 'G','F'])

color_type_one_hot = feature_column.indicator_column(color_type)
demo(color_type_one_hot)


[[0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]]


In [ ]:
#embedding
clarity = feature_column.categorical_column_with_vocabulary_list(
      'clarity', df.clarity.unique())
clarity_embedding = feature_column.embedding_column(clarity, dimension=6)
demo(clarity_embedding)

[[ 0.1763319   0.2434377   0.0428627  -0.0937777   0.05660307  0.32747972]
 [-0.28411624  0.10456126  0.07644629  0.4000848  -0.14661269  0.00801425]
 [ 0.1763319   0.2434377   0.0428627  -0.0937777   0.05660307  0.32747972]
 [ 0.10927127 -0.28786066  0.3216826   0.18922158  0.41692674 -0.44656208]
 [-0.49079388 -0.52965915 -0.4379554  -0.8131934   0.49662971  0.37022093]
 [-0.10639989  0.42744058  0.64257824  0.03294637 -0.2062892  -0.29941705]
 [-0.28411624  0.10456126  0.07644629  0.4000848  -0.14661269  0.00801425]
 [-0.49079388 -0.52965915 -0.4379554  -0.8131934   0.49662971  0.37022093]
 [ 0.602938   -0.39480948  0.09766889  0.10129858 -0.01292356  0.5824666 ]
 [-0.49079388 -0.52965915 -0.4379554  -0.8131934   0.49662971  0.37022093]]


In [ ]:
#hashed feature
clarity_hashed = feature_column.categorical_column_with_hash_bucket(
      'clarity', hash_bucket_size=5)
demo(feature_column.indicator_column(clarity_hashed))

[[0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]]


In [ ]:
#cross feature
#data yang di cross harus berupa string, categorical, atau bucketized
crossed_feature = feature_column.crossed_column([carat_buckets, color_type],
                                                hash_bucket_size=10)
demo(feature_column.indicator_column(crossed_feature))

[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [ ]:
#Pilih feature column mana yang akan digunakan
feature_columns = []

In [ ]:
#embedding column
clarity = feature_column.categorical_column_with_vocabulary_list(
      'clarity', df.clarity.unique())
clarity_embedding = feature_column.embedding_column(clarity, dimension=8)
feature_columns.append(clarity_embedding)

In [ ]:
# numeric column
for header in ['carat', 'depth', 'x', 'y', 'z']:
  feature_columns.append(feature_column.numeric_column(header))

In [ ]:
#membuat feature layer
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [ ]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [ ]:
#create, compile, and train the model
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)

Epoch 1/10
1686/1686 [==============================] - 5s 3ms/step - loss: 0.0043 - accuracy: 0.9989 - val_loss: 4.1922e-04 - val_accuracy: 1.0000
Epoch 2/10
1686/1686 [==============================] - 5s 3ms/step - loss: 4.0433e-04 - accuracy: 1.0000 - val_loss: 4.4770e-04 - val_accuracy: 1.0000
Epoch 3/10
1686/1686 [==============================] - 5s 3ms/step - loss: 4.7875e-04 - accuracy: 1.0000 - val_loss: 4.4775e-04 - val_accuracy: 1.0000
Epoch 4/10
1686/1686 [==============================] - 5s 3ms/step - loss: 5.1438e-04 - accuracy: 1.0000 - val_loss: 4.1451e-04 - val_accuracy: 1.0000
Epoch 5/10
1686/1686 [==============================] - 5s 3ms/step - loss: 4.4862e-04 - accuracy: 1.0000 - val_loss: 4.4255e-04 - val_accuracy: 1.0000
Epoch 6/10
1686/1686 [==============================] - 5s 3ms/step - loss: 4.3492e-04 - accuracy: 1.0000 - val_loss: 4.0237e-04 - val_accuracy: 1.0000
Epoch 7/10
1686/1686 [==============================] - 5s 3ms/step - loss: 3.5005e-04 - acc

In [ ]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

1686/1686 [==============================] - 2s 1ms/step - loss: 3.0858e-04 - accuracy: 1.0000
Accuracy 0.9999814629554749
